## Classify Images of Colon Cancer

isCancerous classification: Binary classification to detect whether a given colon cell image is cancerous or not.

cellType classification: Multi-class classification to categorize the cell type (e.g., fibroblast, epithelial, etc.).

### Step 1: Import Libraries and Set Device

In [2]:
!pip install tqdm --upgrade

In [3]:
import os
import optuna
import random
import pandas as pd
import numpy as np
import torch
import seaborn as sns
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt